In [1]:
import os
import time
import cv2
import numpy as np

In [2]:
# Get List of Images
files = None
for f in os.walk("data/"):
    files = f[2]

print(files)

['00805.jpg', '02222.jpg', '04464.jpg', '06482.jpg', '07209.jpg', '07820.jpg', '07950.jpg', '08089.jpg', '09690.jpg', '20360.jpg', '20825.jpg', '20860.jpg', '25449.jpg', '28038.jpg', '28258.jpg', '29400.jpg', '30879.jpg', '32050.jpg', '33907.jpg', '34005.jpg', '34806.jpg', '37570.jpg', '39073.jpg', '39207.jpg', '39850.jpg', '40073.jpg', '40620.jpg', '40970.jpg', '40985.jpg', '42035.jpg', '43000.jpg', '45326.jpg', '45625.jpg', '46234.jpg', '46323.jpg', '49047.jpg', '49846.jpg', '50303.jpg', '50709.jpg', '50749.jpg', '52547.jpg', '52673.jpg', '53625.jpg', '54703.jpg', '54768.jpg', '54989.jpg', '56385.jpg', '57375.jpg', '67754.jpg', '69009.jpg', '70069.jpg', '70506.jpg', '70980.jpg', '72804.jpg', '73405.jpg', '74205.jpg', '75600.jpg', '76400.jpg', '76768.jpg', '78707.jpg', '79048.jpg', '80632.jpg', '82770.jpg', '84058.jpg', '86907.jpg', '87243.jpg', '87440.jpg', '90205.jpg', '90365.jpg', '90436.jpg', '90480.jpg', '90825.jpg', '93830.jpg', '94242.jpg', '94532.jpg', '98240.jpg', '98900.jpg'

In [3]:
x = []
y = []

In [4]:
dic = {}
for num in range(0,10):
    dic[str(num)] = 0
def processImage(img,filename):
                 
    debug = not True
    kernel = np.ones((2,2),np.uint8)
    bordersize = 10
    kernel2 = np.ones((12,12),np.float32)/225
    


    # Basic Processing
    resize = cv2.resize(img, (400,200))
    border = cv2.copyMakeBorder(
    resize,
    top=bordersize,
    bottom=bordersize,
    left=bordersize,
    right=bordersize,
    borderType=cv2.BORDER_CONSTANT,
    value=[255, 255,255]
    )
    gray = cv2.cvtColor(border,cv2.COLOR_BGR2GRAY)
    smoothed = cv2.filter2D(gray,-1,kernel2)
    retval,thresh = cv2.threshold(smoothed, 30, 255, cv2.THRESH_BINARY)
    erode = cv2.erode(thresh,kernel,iterations = 2)
#     dialate = cv2.dilate(erode,kernel,iterations = 1)
    
    if debug:
        cv2.imshow("Original", img)
        cv2.waitKey(0)
        cv2.imshow("Resized", resize)
        cv2.waitKey(0)
        cv2.imshow("Bordered", border)
        cv2.waitKey(0)
        cv2.imshow("Gray", gray)
        cv2.waitKey(0)
        cv2.imshow("Smooth", smoothed)
        cv2.waitKey(0)
        cv2.imshow("Thresholding", thresh)
        cv2.waitKey(0)
        cv2.imshow("Erode", erode)
        cv2.waitKey(0)
        cv2.imshow("Dilate", dialate)
        cv2.waitKey(0)
    
    final = np.copy(erode)
    
    
    # Find Contours
    contourImg = np.copy(final)
#     contourImg = cv2.cvtColor(contourImg, cv2.COLOR_GRAY2BGR)
    _, contours, hie = cv2.findContours(final, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    
    hie = hie[0]

    goodContours = []

    ind = 0
    for c in contours:        
        testimg = np.copy(contourImg)
        rect = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        x,y,w,h = rect
        cv2.rectangle(testimg,(x,y),(x+w,y+h),(0,255,0),2)
#         cv2.imshow("test", testimg)
        cv2.waitKey(1)
        parent = hie[ind][3]
#         print(area, parent)
#         time.sleep(3)

        ind+=1
        if(parent == 0):
            goodContours.append(c)
#         if(area < 1800 or area > 7000): 
#             continue
        
        
#     print("GoodContours: ", len(goodContours))
    cv2.destroyAllWindows()

    rects = []

    for c in goodContours:
        rect = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        x,y,w,h = rect
#         print(w,h)
        if(w > 150):
            x2 = x + int(w/3)
            x3 = x2 + int(w/3)
            rects.append((x2,y,int(w/3),h))
            rects.append((x3,y,int(w/3),h))
            rects.append((x,y,int(w/3),h))
            cv2.rectangle(contourImg,(x2,y),(x2+int(w/3),y+h),(0,255,0),2)
            cv2.rectangle(contourImg,(x3,y),(x3+int(w/3),y+h),(0,255,0),2)
            cv2.rectangle(contourImg,(x,y),(x+int(w/3),y+h),(0,255,0),2)
        elif (w > 65):
            x2 = x + int(w/2)
            rects.append((x2,y,int(w/2),h))
            rects.append((x,y,int(w/2),h))
            cv2.rectangle(contourImg,(x2,y),(x2+int(w/2),y+h),(0,255,0),2)
            cv2.rectangle(contourImg,(x,y),(x+int(w/2),y+h),(0,255,0),2)
        else:
            rects.append((x,y,w,h))
            cv2.rectangle(contourImg,(x,y),(x+w,y+h),(0,255,0),2)
            
#     cv2.imshow("contour", contourImg)
#     cv2.waitKey(1)
#     time.sleep(1)

#     print(rects)
    rects = sorted(rects)

#     print("Contours:", len(rects))
    if (len(rects) != 5):
        return
        
#     print(dic)

    i = 0
    for (x,y,w,h) in rects:
        digit = filename[i]
        i+=1
#         print(digit)
#         Draw rectangle around contours found
    #     cv2.rectangle(contourImg,(x,y),(x+w,y+h),(0,255,0),2)

    #     Draw the contours on image
    #     cv2.drawContours(contourImg, c, -1, (0, 255, 0), 1)

    #     Region of Interest
        roi = final[y:y+h,x:x+w]
        display = cv2.resize(roi, (120,120))
        roi = cv2.resize(roi, (20,20))
        dic[digit] = dic[digit] + 1 
        cv2.imwrite("digitdata/{}_{}.jpg".format(digit,dic[digit]), roi)
#         print(roi.shape) 
#         pred = clf.predict(np.reshape(roi,(1,64)))
#         print(pred)
    #     print(roi)
        cv2.imshow("roi", display)
#         if digit == '9' and dic[digit] > 24:
#             cv2.waitKey(1000)
#             print(filename)
#         else:
#             cv2.waitKey(1)
        cv2.waitKey(1)
        cv2.destroyWindow(digit)
    #     cv2.imshow("roid", roid)
        
#         time.sleep(0.4)
        
    cv2.destroyAllWindows()

# processImage(cv2.imread("data/04464.jpg"),'04464')

In [5]:
c = 0
for i in files:
    if c > 5:
        break
#     c+=1
    filepath = "data/" + i
    file = cv2.imread(filepath)
    processImage(file, i)
    arr = i.split(".")[0]
    print(arr)
#     print(file.shape)

00805
02222
04464
06482
07209
07820
07950
08089
09690
20360
20825
20860
25449
28038
28258
29400
30879
32050
33907
34005
34806
37570
39073
39207
39850
40073
40620
40970
40985
42035
43000
45326
45625
46234
46323
49047
49846
50303
50709
50749
52547
52673
53625
54703
54768
54989
56385
57375
67754
69009
70069
70506
70980
72804
73405
74205
75600
76400
76768
78707
79048
80632
82770
84058
86907
87243
87440
90205
90365
90436
90480
90825
93830
94242
94532
98240
98900
99503


In [6]:
cv2.destroyAllWindows()